In [1]:
# Styling notebook
from IPython.core.display import display, HTML
def css_styling():
    styles = open("./styles/custom.css", "r").read()
    return HTML(styles)
css_styling()

C:\Users\Renata\AppData\Local\Temp\ipykernel_20712\1617930524.py:2: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# Inicio

## Links uteis 

- Cores em graficos plotly: https://plotly.com/python/discrete-color/
- https://www.selecolor.com/en/recommended-color-palette/

## Importação de libs

In [2]:
# para ler arquivo excel (xls)
# conda install -c anaconda xlrd

import datetime
from datetime import datetime
import os

# Analise
import pandas as pd 
import numpy as np 

#Visualization
import matplotlib
import matplotlib.pyplot as plt 
%matplotlib inline
import seaborn as sns 
import plotly
import plotly.express as px

## Configurações

In [3]:
# para habilitar rolagem horizontal do dataframe 
# caso não funcione no Firefox
from IPython.display import display, HTML
display(HTML("<style>.jp-OutputArea-output {display:flex}</style>"))

In [4]:
# formatação de mensagens de print
class color:
    PURPLE = '\033[95m'
    CYAN = '\033[96m'
    DARKCYAN = '\033[36m'
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    END = '\033[0m'
#print(color.BOLD + 'Hello World !' + color.END)

## Parâmetros

In [5]:
pd.options.display.max_rows = 999
pd.options.display.max_columns = 200
pd.set_option('max_colwidth', 1000) # tamanho das colunas para exibição do conteúdo de textos longos

np.set_printoptions(threshold=1000)

# Formatar numeros float (2 casas decimais)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.float_format', lambda x: '{:.2f}'.format(x))

# Formatações para gráficos
sns.set(style="darkgrid") #cor da area dos plots
figsize = (10,10)

# Definição de funções

## checar_df

Para um determinado dataframe, são verificados:
- total de registros;
- quantidade e tipos de colunas;
- quantidade de valores únicos;
- quantidade de valores nulos (para identificar a necessidade de pre processamento dos dados).

In [6]:
def checar_df(df):
    print(color.BOLD  + color.RED + 'Informações do dataframe: ' + color.END)
    df.info()
    
    print(color.BOLD + color.RED + "\nNúmero de observacoes na base de dados: " + color.END, df.shape[0])
    print('Total de ', df.shape[1], 'Colunas: ', df.columns.values.tolist())
    
    print(color.BOLD + color.RED + "\nNúmero de valores unicos (Não inclui NaN): " + color.END)
    display(df.nunique(dropna=True).to_frame(name='Contagem Unicos').sort_index())
    
    print(color.BOLD + color.RED + "Número de valores nulos:" + color.END)
    display(df.isnull().sum().sort_values(ascending=False).to_frame(name='Contagem Nulos').sort_index())

## define_tp_dimensao

In [32]:
def define_tp_dimensao(x):
    # para simplificar: considerar presencial/ à distância
    if x == 1:   return 'Cursos presenciais'
    elif x == 2: return 'Cursos à distância'
    elif x == 3: return 'Cursos à distância'
    elif x == 4: return 'Cursos à distância'
    else: return 'não informado'

## define_tp_cat_adm

In [64]:
def define_tp_cat_adm(x):
    if x == 1:   return 'Pública Federal'
    elif x == 2: return 'Pública Estadual'
    elif x == 3: return 'Pública Municipal'
    elif x == 4: return 'Privada com fins lucrativos'
    elif x == 5: return 'Privada sem fins lucrativos'
    elif x == 6: return 'Privada - Particular em sentido estrito'
    elif x == 7: return 'Especial'        
    elif x == 8: return 'Privada comunitária'        
    elif x == 9: return 'Privada confessional'        
    else: return 'não informado'

## define_tp_org_acad

In [51]:
def define_tp_org_acad(x):
    if x == 1: return 'Universidade'    
    elif x == 2: return 'Centro Universitário'    
    elif x == 3: return 'Faculdade'
    elif x == 4: return 'Instituto Federal de Educação, Ciência e Tecnologia'
    elif x == 5: return 'Centro Federal de Educação Tecnológica'        
    else: return 'não informado'

## define_tp_org_principal

In [54]:
def define_tp_org_principal(x):
    if x == 1: return 'Faculdade'               #'Universidade'    
    elif x == 2: return 'Centro Universitário'  #'Centro Universitário'    
    elif x == 3: return 'Faculdade'             #'Faculdade'
    elif x == 4: return 'Outros'  # 'Instituto Federal de Educação, Ciência e Tecnologia' 
    elif x == 5: return  'Outros' #'Centro Federal de Educação Tecnológica'        
    else: return 'não informado'

## define_tp_rede

In [55]:
def define_tp_rede(x):
    if x == 1: return 'Pública'  
    elif x == 2: return 'Privada'
    else: return 'não informado'

## define_tp_grau_academico

In [45]:
def define_tp_grau_academico(x):
    if x == 1: return 'Bacharelado'           
    elif x == 2: return 'Licenciatura'  
    elif x == 3: return 'Tecnológico'   
    elif x == 4: return 'Bacharelado e Licenciatura' 
    else: return 'não aplicavel'

## define_tp_mod_ensino

In [46]:
def define_tp_mod_ensino(x):
    if x == 1: return 'Presencial'           
    elif x == 2: return 'A distância'  
    else: return 'não aplicavel'

## define_tp_nivel_acad

In [43]:
def define_tp_nivel_acad(x):
    if x == 1: return 'Graduacao'           
    elif x == 2: return 'Sequencial Formacao Especifica'  
    else: return 'não aplicavel'

# Carga dos Dados 

## Carga dados IES

In [12]:
ies = pd.read_csv('./dados/preparados/dados_ies_consolidado.csv', sep='|', 
                  low_memory=False)

In [13]:
# selecionar todas colunas que começam com CO_ e outras (transformar em string) 
colunas_CO = [col for col in ies if col.startswith('CO_')] + ['COD_IBGE', 'IN_CAPITAL_IES','NU_CEP_IES']
dict_dtype = {column : 'str'  for column in colunas_CO}

ies = pd.read_csv('./dados/preparados/dados_ies_consolidado.csv', sep='|', 
                  dtype = dict_dtype, 
                  low_memory=False)

ies.head(3).T

,0,1,2
NU_ANO_CENSO,2022,2022,2022
NO_REGIAO_IES,Centro-Oeste,Centro-Oeste,Centro-Oeste
CO_REGIAO_IES,5,5,5
NO_UF_IES,Mato Grosso,Mato Grosso,Mato Grosso
SG_UF_IES,MT,MT,MT
CO_UF_IES,51,51,51
NO_MUNICIPIO_IES,Cuiabá,Cuiabá,Cuiabá
IN_CAPITAL_IES,1,1,1
NO_MESORREGIAO_IES,Centro-Sul Mato-grossense,Centro-Sul Mato-grossense,Centro-Sul Mato-grossense
CO_MESORREGIAO_IES,4,4,4


In [9]:
checar_df(ies)

Informações do dataframe: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2595 entries, 0 to 2594
Data columns (total 93 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   NU_ANO_CENSO                       2595 non-null   int64  
 1   NO_REGIAO_IES                      2595 non-null   object 
 2   CO_REGIAO_IES                      2595 non-null   object 
 3   NO_UF_IES                          2595 non-null   object 
 4   SG_UF_IES                          2595 non-null   object 
 5   CO_UF_IES                          2595 non-null   object 
 6   NO_MUNICIPIO_IES                   2595 non-null   object 
 7   IN_CAPITAL_IES                     2595 non-null   object 
 8   NO_MESORREGIAO_IES                 2595 non-null   object 
 9   CO_MESORREGIAO_IES                 2595 non-null   object 
 10  NO_MICRORREGIAO_IES                2595 non-null   object 
 11  CO_MICRORREGIAO_IES          

,Contagem Unicos
CAPITAL,2
COD_IBGE,719
CO_IES,2595
CO_MANTENEDORA,1769
CO_MESORREGIAO_IES,15
CO_MICRORREGIAO_IES,66
CO_REGIAO_IES,5
CO_UF_IES,27
DS_COMPLEMENTO_ENDERECO_IES,893
DS_ENDERECO_IES,2217


Número de valores nulos:


,Contagem Nulos
CAPITAL,0
COD_IBGE,0
CO_IES,0
CO_MANTENEDORA,0
CO_MESORREGIAO_IES,0
CO_MICRORREGIAO_IES,0
CO_REGIAO_IES,0
CO_UF_IES,0
DS_COMPLEMENTO_ENDERECO_IES,1
DS_ENDERECO_IES,0


## Carga Dados Cursos

In [25]:
%%time
cursos_orig = pd.read_csv('./dados/MICRODADOS_CADASTRO_CURSOS_2022.CSV', 
                          #dtype = dict_dtype,
                          sep=';',
                          encoding='iso-8859-1',
                          low_memory=False)

print('Total registros: ', cursos_orig.shape[0])
print('Total colunas: ', cursos_orig.shape[1])

Total registros:  573019
Total colunas:  200
CPU times: total: 13 s
Wall time: 13 s


In [26]:
# selecionar todas colunas que começam com CO_ e outras (transformar em string) 
# facilita operações de merge (tem que conter os mesmos tipos de dados)
colunas_CO = [col for col in cursos_orig if col.startswith('CO_')] + ['IN_CAPITAL','IN_GRATUITO']
dict_dtype = {column : 'str'  for column in colunas_CO}

cursos_orig = pd.read_csv('./dados/MICRODADOS_CADASTRO_CURSOS_2022.CSV', 
                          dtype = dict_dtype,
                          sep=';',
                          encoding='iso-8859-1',
                          low_memory=False)

print('Total registros: ', cursos_orig.shape[0])
print('Total colunas: ', cursos_orig.shape[1])

Total registros:  573019
Total colunas:  200


In [19]:
cursos_orig.info(max_cols=500)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 573019 entries, 0 to 573018
Data columns (total 200 columns):
 #    Column                        Non-Null Count   Dtype 
---   ------                        --------------   ----- 
 0    NU_ANO_CENSO                  573019 non-null  int64 
 1    NO_REGIAO                     563416 non-null  object
 2    CO_REGIAO                     573019 non-null  object
 3    NO_UF                         563416 non-null  object
 4    SG_UF                         563416 non-null  object
 5    CO_UF                         573019 non-null  object
 6    NO_MUNICIPIO                  563416 non-null  object
 7    CO_MUNICIPIO                  573019 non-null  object
 8    IN_CAPITAL                    573019 non-null  object
 9    TP_DIMENSAO                   573019 non-null  int64 
 10   TP_ORGANIZACAO_ACADEMICA      573019 non-null  int64 
 11   TP_CATEGORIA_ADMINISTRATIVA   573019 non-null  int64 
 12   TP_REDE                       573019 non-n

In [18]:
checar_df(cursos_orig)

Informações do dataframe: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 573019 entries, 0 to 573018
Columns: 200 entries, NU_ANO_CENSO to QT_CONC_MOB_ACADEMICA
dtypes: int64(180), object(20)
memory usage: 874.4+ MB

Número de observacoes na base de dados:  573019
Total de  200 Colunas:  ['NU_ANO_CENSO', 'NO_REGIAO', 'CO_REGIAO', 'NO_UF', 'SG_UF', 'CO_UF', 'NO_MUNICIPIO', 'CO_MUNICIPIO', 'IN_CAPITAL', 'TP_DIMENSAO', 'TP_ORGANIZACAO_ACADEMICA', 'TP_CATEGORIA_ADMINISTRATIVA', 'TP_REDE', 'CO_IES', 'NO_CURSO', 'CO_CURSO', 'NO_CINE_ROTULO', 'CO_CINE_ROTULO', 'CO_CINE_AREA_GERAL', 'NO_CINE_AREA_GERAL', 'CO_CINE_AREA_ESPECIFICA', 'NO_CINE_AREA_ESPECIFICA', 'CO_CINE_AREA_DETALHADA', 'NO_CINE_AREA_DETALHADA', 'TP_GRAU_ACADEMICO', 'IN_GRATUITO', 'TP_MODALIDADE_ENSINO', 'TP_NIVEL_ACADEMICO', 'QT_CURSO', 'QT_VG_TOTAL', 'QT_VG_TOTAL_DIURNO', 'QT_VG_TOTAL_NOTURNO', 'QT_VG_TOTAL_EAD', 'QT_VG_NOVA', 'QT_VG_PROC_SELETIVO', 'QT_VG_REMANESC', 'QT_VG_PROG_ESPECIAL', 'QT_INSCRITO_TOTAL', 'QT_INSCRITO_T

,Contagem Unicos
CO_CINE_AREA_DETALHADA,89
CO_CINE_AREA_ESPECIFICA,37
CO_CINE_AREA_GERAL,11
CO_CINE_ROTULO,351
CO_CURSO,45296
CO_IES,2595
CO_MUNICIPIO,3310
CO_REGIAO,6
CO_UF,28
IN_CAPITAL,2


Número de valores nulos:


,Contagem Nulos
CO_CINE_AREA_DETALHADA,0
CO_CINE_AREA_ESPECIFICA,0
CO_CINE_AREA_GERAL,0
CO_CINE_ROTULO,0
CO_CURSO,0
CO_IES,0
CO_MUNICIPIO,0
CO_REGIAO,0
CO_UF,0
IN_CAPITAL,0


In [47]:
cursos_orig.sample(5)

,NU_ANO_CENSO,NO_REGIAO,CO_REGIAO,NO_UF,SG_UF,CO_UF,NO_MUNICIPIO,CO_MUNICIPIO,IN_CAPITAL,TP_DIMENSAO,TP_ORGANIZACAO_ACADEMICA,TP_CATEGORIA_ADMINISTRATIVA,TP_REDE,CO_IES,NO_CURSO,CO_CURSO,NO_CINE_ROTULO,CO_CINE_ROTULO,CO_CINE_AREA_GERAL,NO_CINE_AREA_GERAL,CO_CINE_AREA_ESPECIFICA,NO_CINE_AREA_ESPECIFICA,CO_CINE_AREA_DETALHADA,NO_CINE_AREA_DETALHADA,TP_GRAU_ACADEMICO,IN_GRATUITO,TP_MODALIDADE_ENSINO,TP_NIVEL_ACADEMICO,QT_CURSO,QT_VG_TOTAL,QT_VG_TOTAL_DIURNO,QT_VG_TOTAL_NOTURNO,QT_VG_TOTAL_EAD,QT_VG_NOVA,QT_VG_PROC_SELETIVO,QT_VG_REMANESC,QT_VG_PROG_ESPECIAL,QT_INSCRITO_TOTAL,QT_INSCRITO_TOTAL_DIURNO,QT_INSCRITO_TOTAL_NOTURNO,QT_INSCRITO_TOTAL_EAD,QT_INSC_VG_NOVA,QT_INSC_PROC_SELETIVO,QT_INSC_VG_REMANESC,QT_INSC_VG_PROG_ESPECIAL,QT_ING,QT_ING_FEM,QT_ING_MASC,QT_ING_DIURNO,QT_ING_NOTURNO,QT_ING_VG_NOVA,QT_ING_VESTIBULAR,QT_ING_ENEM,QT_ING_AVALIACAO_SERIADA,QT_ING_SELECAO_SIMPLIFICA,QT_ING_EGR,QT_ING_OUTRO_TIPO_SELECAO,QT_ING_PROC_SELETIVO,QT_ING_VG_REMANESC,QT_ING_VG_PROG_ESPECIAL,QT_ING_OUTRA_FORMA,QT_ING_0_17,QT_ING_18_24,QT_ING_25_29,QT_ING_30_34,QT_ING_35_39,QT_ING_40_49,QT_ING_50_59,QT_ING_60_MAIS,QT_ING_BRANCA,QT_ING_PRETA,QT_ING_PARDA,QT_ING_AMARELA,QT_ING_INDIGENA,QT_ING_CORND,QT_MAT,QT_MAT_FEM,QT_MAT_MASC,QT_MAT_DIURNO,QT_MAT_NOTURNO,QT_MAT_0_17,QT_MAT_18_24,QT_MAT_25_29,QT_MAT_30_34,QT_MAT_35_39,QT_MAT_40_49,QT_MAT_50_59,QT_MAT_60_MAIS,QT_MAT_BRANCA,QT_MAT_PRETA,QT_MAT_PARDA,QT_MAT_AMARELA,QT_MAT_INDIGENA,QT_MAT_CORND,QT_CONC,QT_CONC_FEM,QT_CONC_MASC,QT_CONC_DIURNO,QT_CONC_NOTURNO,QT_CONC_0_17,QT_CONC_18_24,QT_CONC_25_29,QT_CONC_30_34,QT_CONC_35_39,QT_CONC_40_49,QT_CONC_50_59,QT_CONC_60_MAIS,QT_CONC_BRANCA,QT_CONC_PRETA,QT_CONC_PARDA,QT_CONC_AMARELA,QT_CONC_INDIGENA,QT_CONC_CORND,QT_ING_NACBRAS,QT_ING_NACESTRANG,QT_MAT_NACBRAS,QT_MAT_NACESTRANG,QT_CONC_NACBRAS,QT_CONC_NACESTRANG,QT_ALUNO_DEFICIENTE,QT_ING_DEFICIENTE,QT_MAT_DEFICIENTE,QT_CONC_DEFICIENTE,QT_ING_FINANC,QT_ING_FINANC_REEMB,QT_ING_FIES,QT_ING_RPFIES,QT_ING_FINANC_REEMB_OUTROS,QT_ING_FINANC_NREEMB,QT_ING_PROUNII,QT_ING_PROUNIP,QT_ING_NRPFIES,QT_ING_FINANC_NREEMB_OUTROS,QT_MAT_FINANC,QT_MAT_FINANC_REEMB,QT_MAT_FIES,QT_MAT_RPFIES,QT_MAT_FINANC_REEMB_OUTROS,QT_MAT_FINANC_NREEMB,QT_MAT_PROUNII,QT_MAT_PROUNIP,QT_MAT_NRPFIES,QT_MAT_FINANC_NREEMB_OUTROS,QT_CONC_FINANC,QT_CONC_FINANC_REEMB,QT_CONC_FIES,QT_CONC_RPFIES,QT_CONC_FINANC_REEMB_OUTROS,QT_CONC_FINANC_NREEMB,QT_CONC_PROUNII,QT_CONC_PROUNIP,QT_CONC_NRPFIES,QT_CONC_FINANC_NREEMB_OUTROS,QT_ING_RESERVA_VAGA,QT_ING_RVREDEPUBLICA,QT_ING_RVETNICO,QT_ING_RVPDEF,QT_ING_RVSOCIAL_RF,QT_ING_RVOUTROS,QT_MAT_RESERVA_VAGA,QT_MAT_RVREDEPUBLICA,QT_MAT_RVETNICO,QT_MAT_RVPDEF,QT_MAT_RVSOCIAL_RF,QT_MAT_RVOUTROS,QT_CONC_RESERVA_VAGA,QT_CONC_RVREDEPUBLICA,QT_CONC_RVETNICO,QT_CONC_RVPDEF,QT_CONC_RVSOCIAL_RF,QT_CONC_RVOUTROS,QT_SIT_TRANCADA,QT_SIT_DESVINCULADO,QT_SIT_TRANSFERIDO,QT_SIT_FALECIDO,QT_ING_PROCESCPUBLICA,QT_ING_PROCESCPRIVADA,QT_ING_PROCNAOINFORMADA,QT_MAT_PROCESCPUBLICA,QT_MAT_PROCESCPRIVADA,QT_MAT_PROCNAOINFORMADA,QT_CONC_PROCESCPUBLICA,QT_CONC_PROCESCPRIVADA,QT_CONC_PROCNAOINFORMADA,QT_PARFOR,QT_ING_PARFOR,QT_MAT_PARFOR,QT_CONC_PARFOR,QT_APOIO_SOCIAL,QT_ING_APOIO_SOCIAL,QT_MAT_APOIO_SOCIAL,QT_CONC_APOIO_SOCIAL,QT_ATIV_EXTRACURRICULAR,QT_ING_ATIV_EXTRACURRICULAR,QT_MAT_ATIV_EXTRACURRICULAR,QT_CONC_ATIV_EXTRACURRICULAR,QT_MOB_ACADEMICA,QT_ING_MOB_ACADEMICA,QT_MAT_MOB_ACADEMICA,QT_CONC_MOB_ACADEMICA
248314,2022,Sudeste,3,Minas Gerais,MG,31,Divinópolis,3122306,0,2,2,4,2,1270,Banco De Dados,1484996,Banco de dados,"""0612B01""",06,Computação e Tecnologias da Informação e Comunicação (TIC),061,Computação e Tecnologias da Informação e Comunicação (TIC),0612,Infraestrutura e gestão de TIC,3,0,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,2,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,2,0,0,0,0,2,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [59]:
#col_interesse = []
#cursos = cursos_orig[col_interesse].copy()
cursos = cursos_orig.copy()

## Carga dados Indicadores Municipais (Atlas) - 2010

In [27]:
# definir os tipos de algumas colunas
list_col_string = ['Codmun6','Codmun7'] 
dict_dtype = {column : 'str'  for column in list_col_string}

df_ind_atlas = pd.read_csv('./dados/selecao_indicadores_sociais_municipios.csv', 
                       sep='|', dtype = dict_dtype, low_memory=False) 

# considerar apenas a ultima referencia (2010)
df_ind_atlas = df_ind_atlas[df_ind_atlas['ANO']==2010]

col_excluir = ['ANO']
for c in col_excluir: del df_ind_atlas[c]

# renomear colunas 
df_ind_atlas = df_ind_atlas.rename(columns = {
                                    'Codmun6':'CodIBGE_6',
                                    'Codmun7':'CodIBGE',
                                    'pesotot':'Pop_total',
                                    'pesourb':'Pop_urbana',
                                    'POP':'Pop_total_dom_part',    
                                    'POPT':'Pop_total_dom_part_exc_renda_nula'    
                                })

In [28]:
col_relevantes = ['CodIBGE', 'Pop_total','Pop_urbana','Pop_total_dom_part', 'Pop_total_dom_part_exc_renda_nula']
df_ind_atlas[col_relevantes].head(5).T

,11130,11131,11132,11133,11134
CodIBGE,1100015,1100023,1100031,1100049,1100056
Pop_total,24392,90353,6313,78574,17029
Pop_urbana,13970,76525,2693,61921,14419
Pop_total_dom_part,24097,89766,6303,78141,16942
Pop_total_dom_part_exc_renda_nula,22429,88730,6156,76876,16815


In [29]:
df_ind_atlas.shape

(5565, 111)

In [24]:
# checar as diferentes infos de Pop de um municipio
# display(ibge[ibge['NOME_MUNICIPIO']=='Brasília'])

# col = ['Município','Pop_total','Pop_urbana','Pop_total_dom_part','Pop_total_dom_part_exc_renda_nula']
# df_ind_atlas[df_ind_atlas['CodIBGE']=='5300108'][col]

# Tratamento dados 

## Enriquecimento de dados

### Adicionar Descrição do Tipo de Dimensão

In [60]:
# Convertendo coluna TP_DIMENSAO
cursos.insert(cursos.columns.get_loc('TP_DIMENSAO')+1, \
                'Tipo_Dimensao', cursos['TP_DIMENSAO'].apply(lambda x: define_tp_dimensao(x)))

### Adicionar Descrição Tipo Organização Acadêmica

In [61]:
# Convertendo coluna TP_ORGANIZACAO_ACADEMICA
cursos.insert(cursos.columns.get_loc('TP_ORGANIZACAO_ACADEMICA')+1, \
                'Tipo_Org_Acad', cursos['TP_ORGANIZACAO_ACADEMICA'].apply(lambda x: define_tp_org_acad(x)))

### Adicionar Organizacao principal: Faculdade, Centro e Outros

In [62]:
# Simplificando o conceito de Organização administrativa
cursos.insert(cursos.columns.get_loc('Tipo_Org_Acad')+1, \
                'Tipo_Org_Principal', cursos['TP_ORGANIZACAO_ACADEMICA'].apply(lambda x: define_tp_org_principal(x)))

### Adicionar Descrição Tipo Categoria Administrativa

In [65]:
# Convertendo coluna TP_CATEGORIA_ADMINISTRATIVA
cursos.insert(cursos.columns.get_loc('TP_CATEGORIA_ADMINISTRATIVA')+1, \
                'Tipo_Cat_Admn', cursos['TP_CATEGORIA_ADMINISTRATIVA'].apply(lambda x: define_tp_cat_adm(x)))

### Adicionar Descrição Tipo Rede

In [66]:
# Convertendo coluna TP_REDE
cursos.insert(cursos.columns.get_loc('TP_REDE')+1, \
                'Tipo_Rede', cursos['TP_REDE'].apply(lambda x: define_tp_rede(x)))

### Adicionar Descrição Tipo Grau Academico

In [67]:
# Convertendo coluna TP_GRAU_ACADEMICO
cursos.insert(cursos.columns.get_loc('TP_GRAU_ACADEMICO')+1, \
                'Tipo_Grau_Acad', cursos['TP_GRAU_ACADEMICO'].apply(lambda x: define_tp_grau_academico(x)))

### Adicionar Descrição Tipo Modalidade de Ensino

In [68]:
# Convertendo coluna TP_MODALIDADE_ENSINO
cursos.insert(cursos.columns.get_loc('TP_MODALIDADE_ENSINO')+1, \
                'Tipo_Mod_Ens', cursos['TP_MODALIDADE_ENSINO'].apply(lambda x: define_tp_mod_ensino(x)))

### Adicionar Descrição Nivel Academico

In [69]:
# Convertendo coluna TP_NIVEL_ACADEMICO
cursos.insert(cursos.columns.get_loc('TP_NIVEL_ACADEMICO')+1, \
                'Tipo_Niv_Acad', cursos['TP_NIVEL_ACADEMICO'].apply(lambda x: define_tp_nivel_acad(x)))

## Filtrar registros de interesse

filtrar para 
- Cursos presenciais	- TP_DIMENSAO = 1
- Org Academica			- TP_ORGANIZACAO_ACADEMICA = todos
- Publ/ Privada			- TP_CATEGORIA_ADMINISTRATIVA = todos
- Publ/ Privada			- TP_REDE = todos
- Cursos de NS 			- TP_GRAU_ACADEMICO in (1,2,3,4)
- Cursos presenciais 	- TP_MODALIDADE_ENSINO = 1
- Somente graduacao		- TP_NIVEL_ACADEMICO = 1

In [72]:
print('Total de registros - cursos:', cursos.shape[0])

Total de registros - cursos: 573019


In [73]:
# checar tipos dos dados
col = ['TP_DIMENSAO', 'TP_ORGANIZACAO_ACADEMICA', 'TP_CATEGORIA_ADMINISTRATIVA', 'TP_REDE',
      'TP_GRAU_ACADEMICO', 'TP_MODALIDADE_ENSINO', 'TP_NIVEL_ACADEMICO', ]
cursos[col].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 573019 entries, 0 to 573018
Data columns (total 7 columns):
 #   Column                       Non-Null Count   Dtype
---  ------                       --------------   -----
 0   TP_DIMENSAO                  573019 non-null  int64
 1   TP_ORGANIZACAO_ACADEMICA     573019 non-null  int64
 2   TP_CATEGORIA_ADMINISTRATIVA  573019 non-null  int64
 3   TP_REDE                      573019 non-null  int64
 4   TP_GRAU_ACADEMICO            573019 non-null  int64
 5   TP_MODALIDADE_ENSINO         573019 non-null  int64
 6   TP_NIVEL_ACADEMICO           573019 non-null  int64
dtypes: int64(7)
memory usage: 30.6 MB


In [83]:
display(cursos[['TP_DIMENSAO','Tipo_Dimensao']].value_counts())
cursos[['TP_DIMENSAO','Tipo_Dimensao']].value_counts(normalize=True)

TP_DIMENSAO  Tipo_Dimensao     
2            Cursos à distância    527328
1            Cursos presenciais     36089
3            Cursos à distância      9207
4            Cursos à distância       395
Name: count, dtype: int64

TP_DIMENSAO  Tipo_Dimensao     
2            Cursos à distância   0.92
1            Cursos presenciais   0.06
3            Cursos à distância   0.02
4            Cursos à distância   0.00
Name: proportion, dtype: float64

In [84]:
display(cursos[['TP_GRAU_ACADEMICO','Tipo_Grau_Acad']].value_counts())
cursos[['TP_GRAU_ACADEMICO','Tipo_Grau_Acad']].value_counts(normalize=True)

TP_GRAU_ACADEMICO  Tipo_Grau_Acad
3                  Tecnológico       270752
1                  Bacharelado       183827
2                  Licenciatura      117033
0                  não aplicavel       1407
Name: count, dtype: int64

TP_GRAU_ACADEMICO  Tipo_Grau_Acad
3                  Tecnológico      0.47
1                  Bacharelado      0.32
2                  Licenciatura     0.20
0                  não aplicavel    0.00
Name: proportion, dtype: float64

In [85]:
display(cursos[['TP_MODALIDADE_ENSINO','Tipo_Mod_Ens']].value_counts())
cursos[['TP_MODALIDADE_ENSINO','Tipo_Mod_Ens']].value_counts(normalize=True)

TP_MODALIDADE_ENSINO  Tipo_Mod_Ens
2                     A distância     536930
1                     Presencial       36089
Name: count, dtype: int64

TP_MODALIDADE_ENSINO  Tipo_Mod_Ens
2                     A distância    0.94
1                     Presencial     0.06
Name: proportion, dtype: float64

In [86]:
display(cursos[['TP_NIVEL_ACADEMICO','Tipo_Niv_Acad']].value_counts())
cursos[['TP_NIVEL_ACADEMICO','Tipo_Niv_Acad']].value_counts(normalize=True)

TP_NIVEL_ACADEMICO  Tipo_Niv_Acad                 
1                   Graduacao                         573010
2                   Sequencial Formacao Especifica         9
Name: count, dtype: int64

TP_NIVEL_ACADEMICO  Tipo_Niv_Acad                 
1                   Graduacao                        1.00
2                   Sequencial Formacao Especifica   0.00
Name: proportion, dtype: float64

In [74]:
cursos_escopo = cursos[
                (cursos['TP_DIMENSAO']== 1 ) & 
                (cursos['TP_GRAU_ACADEMICO'].isin([1,2,3,4]) ) & 
                (cursos['TP_MODALIDADE_ENSINO']==1) & 
                (cursos['TP_NIVEL_ACADEMICO']==1)] 

In [75]:
print('Total de registros - cursos_escopo:', cursos_escopo.shape[0])

Total de registros - cursos_escopo: 35765


In [ ]:
# retirada de cursos com vagas = 0


## Salvar dados Cursos consolidados em CSV

In [76]:
# salvar em arquivo
# cursos.to_csv('./dados/preparados/dados_cursos_consolidado.csv', sep='|', index=False)
# cursos_escopo.to_csv('./dados/preparados/dados_cursos_escopo_consolidado.csv', sep='|', index=False)